# 연쇄 법칙과 역전파

- 역전파 : 국소적인 미분을 순방향과는 방대인 오른쪽에서 왼쪽으로 전달한다. 또한, 이런 국소적 미분을 전달하는 원리는 연쇄 법칙에 따른 것이다. 
  - 연쇄 법칙 : 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다. 이것이 연쇄 법칙의 원리이다. 
  - $\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x}$
- 역전파의 계산 절차는 신호 E에 노드의 국소적 미분을 곱한 후 다음 노드로 전달하는 것이다. 여기에서 말하는 국소적 미분은 순전파 때의 y=f(x) 계산의 미분을 구한다는 것이며, 이는 x에대한 y의 미분을 구한다는 뜻이다. 

- 덧셈 노드의 역전파
  - 역전파 때는 상류에서 전해진 미분에 1을 곱하여 하류로 흘린다. 즉, 덧셈 노드의 역전파는 1을 곱하기만 할 뿐이므로 입력된 값을 그대로 다음 노드로 보내게 된다. 
- 곱셈 노드의 역전파
  - 상류의 값에 순전파 때의 입력 신호들을 서로 바꾼 값을 곱해서 하류로 보낸다. 서로 바꾼 값이란 순전파 때 x였다면 역전파에서는 y, 순전파 때 y였다면 역전파에서는 x로 바꾼다는 의미이다.
  
> 덧셈의 역전파에서는 상류의 값을 그대로 흘려보내서 순방향 입력 신호의 값을 필요하지 않았지만, 곱셈의 역전파는 순방향 입력 신호의 값이 필요하다. 그래서 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장해둔다.

- 앞서 들었던 사과 쇼핑의 예를 들어보자
  - 이 문제에서는 사과의 가격, 사과의 개수, 소비세라는 세 변수 각각이 최종 금액에 어떻게 영향을 주느냐를 풀고자 한다. 이는 사과 가격에 대한 지불 금액의 미분, 사과 개수에 대한 지불 금액의 미분, 소비세에 대한 지불 금액의 미분을 구하는 것에 해당한다. 
  - 앞서 곱셈 노드의 역전파에서는 입력 신호를 서로 바꿔서 하류로 흘린다고 했다. 결과를 보면 사과 과격의 미분은 2.2, 사과 개수의 미분은 110, 소비세의 미분은 200이다. 이는 소비세와 사과 과격이 같은 양만큼 오르면 최종 금액에는 소비세가 200의 크기로 사과 과격이 2.2 크기로 영향을 준다고 해석할 수 있다. 단, 단위는 조심하여야 한다. 

---

- 지금까지 보아온 사과 쇼핑의 예를 파이썬으로 구현해보자

- 먼저, 곱셈 계층부터 구현해보자

In [1]:
class MulLayer: # 곱셈 노드
    def __init__(self): # 생성자 함수로써 인스턴스 변수인 x와 y를 초기화한다. 
        self.x = None
        self.y = None

    def forward(self, x, y): # 순전파
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout): # 역전파
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

- 순전파에서는 x와 y를 인수로 받고 두 값을 곱해서 반환한다. 반면 역전파에서는 상류에서 넘어온 미분(dout)에 순전파 때의 값을 서로 바꿔 곱한 후 하류로 흘려보낸다. 
- 해당 클래스를 이용하여 사과 쇼핑 예시에서의 순전파를 구현해보자

In [3]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(int(price))

220


- 각 변수에 대한 미분은 backward()에서 구할 수 있다.

In [4]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

dApple: 2.2
dApple_num: 110
dTax: 200


- backward() 호출 순서는 forward()때와는 반대이다. 또 backward()가 받는 인수는 순전파의 출력에 대한 미분이다. 가령 mul_apple_layer()라는 곱셈 계층은 순전파 때는 apple_price를 반환하지만 역전파 때는 apple_price의 미분 값인 dapple_price를 인수로 받는다. 

---

- 덧셈 계층을 구현해보자

In [5]:
class AddLayer:
    def __init__(self):
        pass # 덧셈 계층에서는 초기화가 필요없으니 pass처리 한다.
    
    def forward(self,x,y):
        out = x+y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx,dy

- 전체적인 계산 그래프를 파이썬으로 구현해보자

In [6]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


---

- 활성화 함수 계층을 구현해보자, 계산 그래프를 신경망에 적용해보자
  - ReUL 계층, 165p 참고.

In [7]:
class Relu:
    def __init__(self):
        self.mask = None
        # 해당 클래스는 mask라는 인스턴스 변수를 갖는다. bool형 넘파이 배열로 사용될 것이다.
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    def backward(self,dout):
        dout[self.mask] = 0
        dx = sout
        
        return dx

In [10]:
import numpy as np
x = np.array([[1,-0.5],[-2,3]])
print(x)
print('\n')
mask = x<=0
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]


[[False  True]
 [ True False]]


- 이제 sigmoid 함수를 구현해보자
  - +와 x노드 말고도 exp와 /노드도 필요하다.
  - 자세한 과정은 167p부터 169p를 참고하자

In [11]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / ( 1 + np.exp(-x))
        self.out = out
        
    def backward(self, dout):
        dx = dout * (1 - self.out) * self.out
        
        return dx

- 해당 구현에서는 순전파의 출력을 인스턴스 변수 out에 보관했다가, 역전파 계산 때 그 값을 사용하는 것이다.